In [1]:
import os
import comFuncs
from comFuncs import *
import comFuncs3

import string
import rarfile
import re
from multiprocessing import Pool
import time
import resource
import functools

In [2]:
try:
    ind=load_obj('datas/indexdB')
except:
    names=!find /media/some/sam/dall -type f -iname "index*2016*"
    names=names+["/media/some/sam/dall/New ebook packs for August 2016/New ebook packs INDEX 2003-2015.txt"]
    lst=[list(reversed(os.path.split(f))) for f in names]
    result = {}
    for item in lst:
        if not item[0] in result.keys():
            result[item[0]]=[]  
        result[item[0]]=result[item[0]]+ [item[1]]

    ind=[]
    for k in result.keys():
        rd=open(result[k][0]+"/"+k).readlines()
        ind=ind+ zip(rd,[result[k][0]]*len(rd),[k]*len(rd))
    ind=[idc for idc in ind if idc[0].count('\\')>0]
    save_obj(ind, 'datas/indexdB')  
len(ind)

573313

In [3]:
def removeLTdash(strg,charr):
    if len (strg)>3:
        strgg=strg.strip()
        if strgg[0]==charr:
            strgg=strgg[1:].strip()
        if strgg[-1]==charr:
            strgg=strgg[:-1].strip()
        return strgg
    return strg

In [4]:
def extractSeries(strg):
    series=re.findall('.+-(\s*\[.+\]\s*-).+.rar',strg)
    if len(series)>0:
        series=removeLTdash(series[0],'-')
        series=removeLTdash(series,'[')
        series=removeLTdash(series,']')
        return series
    return ""

In [5]:
def extractAuthor(strg):
    author=re.findall('(.+?\s*-)',strg)
    if len(author)>0:
        return removeLTdash(author[0],'-')
    return ""

In [6]:
def extractAdd(title):
    add=[]
    if title.count(')')>0 and title.count('(')>0:
        cnt=0
        while min([title.count('('),title.count(')')])>0:
        
            add.append(re.findall('.*\(\s*(.+?)\s*\)',title)[0] .strip())
        
            title=title.replace(add[-1],"")
            title=re.sub(' +',' ',title)
            cnti=0
            while title.count('()')>0:
                title=title.replace("()","").strip()
                if cnti>9:
                    return title
                cnti+=1
            #print title
            cntj=0   
            while title.count('( )'):
                title=title.replace("( )","").strip()
                if cntj>9:
                    return title,add
                cntj+=1
            if cnt>9:
                return title,add
            cnt+=1
    return title,add

In [7]:
def extractTitle(strg):
    title=re.findall('.+(\s*-.+)\(.*',strg)
    if len(title)>0:
        title=removeLTdash(title[0],'-')
        return title
    return ""

In [8]:
def extractExt(strg):
    ext=re.findall('.+(\s\(.+\)\s*).rar',strg)
    if len(ext)>0:
        return  ''.join(c for c in ext[0] if c in string.letters+string.digits+" ").strip()
    return ""

In [9]:
def extractRecord(inn):
        tmp = inn.strip().split('\\')
        rar=''.join(t for t in tmp[0] if t in string.digits+"- ")
        rar=re.sub(' +',' ',rar)
        if len(tmp)>1:
            series = extractSeries(tmp[1])
            author = extractAuthor(tmp[1])
            title, add = extractAdd(extractTitle(tmp[1]))
            exten = extractExt(tmp[1])
            strg=tmp[1].replace('.rar','')
            strg=strg.replace(author,'')
            strg=strg.replace(title,'')
            strg=strg.replace(series,'')
            strg=strg.replace(exten,'')
            for l in add:
                strg=strg.replace(l,'')
            rem=''.join(st for st in strg if st in string.letters+" '")
            rem=' '.join(rem.split())
            #print 'remaining: ',strg
            #rar folder, file,extra
            return [rar,inn,rem,author,series,title,' '.join(add),exten]
           

In [10]:
indd=zip(*ind)[0]
indd[0]

'03-01-2016 Part 1-2\\A J Bromley - [Annie Parker Mystery Short] - Welcome to Sweetwater (Christmas Cookies; Spring Fling; Shopping Spree; Slim Gym; Afternoon Delight) (retail) (azw3).rar\r\n'

In [11]:
ind[0]

('03-01-2016 Part 1-2\\A J Bromley - [Annie Parker Mystery Short] - Welcome to Sweetwater (Christmas Cookies; Spring Fling; Shopping Spree; Slim Gym; Afternoon Delight) (retail) (azw3).rar\r\n',
 '/media/some/sam/dall/New ebook packs for March 2016',
 'Index 2016-03.txt')

In [12]:
t0 = time.time()
try:
    dBB=load_obj('datas/dBB')
except:
    
    p = Pool(8)
    dBB=p.map(extractRecord, indd)
    p.terminate()
    p.close()
    dBB=filter(None,dBB)
    save_obj(dBB, 'datas/dBB') 
t1 = time.time()
total = t1-t0
print('records: ',len(dBB))
print('time: ',total)
print('resources: ',resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1000)

('records: ', 573313)
('time: ', 52.696621894836426)
('resources: ', 877)


In [13]:
def matchRec(rec,search):

    cos=[]
    for r in rec[2:]:
        if len(r)>2:
            for se in search.split(' '):
                if len(se)>1:
                    cc=get_cosine(text_to_vector(r.lower()),text_to_vector(se.lower()))
                    #print r,se
                    if cc>0.0:
                        cos.append(cc)
    return cos

In [14]:

def findrec(rec,search):
    res=matchRec(rec,search)
    if len(res)>0:
        score=sum(res)/len(search.split(' '))
        #print rec,score,res
        
        if score>.5:
            rt=rec+[score]+res
            print ' '.join( rt[2:6])
            return rt

In [15]:
#serial
#search="powerone"
#t0 = time.time()

#for rec in dBB:
#    findrec(rec,search)
#t1 = time.time()
#t1-t0

In [16]:
#parallel
search="powerone hospital"
t0 = time.time()
p = Pool(20)
res=p.map(functools.partial(findrec, search=search), dBB)
p.terminate()
p.close()
res=filter(None,res)
#save_obj(res, 'datas/res') 
t1 = time.time()
total = t1-t0
print('records: ',len(res))
print('time: ',total)
print('resources: ',resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1000)

 Powerone  Hell Hospital [MF]
('records: ', 1)
('time: ', 121.86885404586792)
('resources: ', 873)


In [81]:
search="Building Harlequins Moon Larry Niven".split()

In [82]:
allres=[list(
        filter(
            lambda x: any(xx.lower().replace("'","").count(se.strip().lower()) > 0 for xx in x[3:7]  ), dBB)) 
        for se in search]

In [86]:
def searchdb(seTerm,db):
    allres=[list
            (filter
             (lambda x: any(xx.lower().replace("'","").count(se.strip().lower()) > 0 for xx in x[3:7]  ), db)) 
            for se in seTerm]
    return reduce(lambda sx, sy: filter(lambda x: x in sx,sy), ((i) for i in allres))

In [25]:
def findr(name, path):
    #print name,path
    res=!find $path -type f -iname $name
    return res

In [26]:
def similarity(a,b):
    return get_cosine(text_to_vector(a.lower()),text_to_vector(b.lower()))

In [120]:
foo="/home/some/an/"
!mkdir "$foo"
cnt=0
sor = sorted(res, key=lambda tup: tup[7])
#epub first
sorr=list(filter(lambda x: x[7].lower().count('epub') > 0, sor))+list(filter(lambda x: x[7].lower().count('epub') == 0, sor))
books=[]
for r in sorr:
 
        found=r[0]
        baseName=' '.join(ss for ss in r[3:6] if len(ss)>0).lower()
        if any(similarity(baseName,b.lower())>.75 for b in books):
            continue
        found=r[0].replace("  "," ").replace(" ","*").replace("_","*")+"*.rar"
        fls=findr(found,'/media/some/sam/dall/')
        #print found
        if len(fls)>0:
            for flss in fls:
                rf = rarfile.RarFile(flss)
                booFound=False
                for f in rf.infolist():
                    #print f.filename, f.file_size
                    if similarity(f.filename,r[1])>0.99:
                        cnt+=1
                        booFound=True
                        print cnt,"- ",f.filename,baseName
                        try:
                            rf.extract(f,foo)
                            books.append(baseName)
                        except:
                            print "reading problem",r[1],""
                        ft=foo+f.filename.split('\\')[1]
                        fo=foo+f.filename.split('\\')[0]
                        fn=fo+"/"+f.filename.split('\\')[1]
                        os.rename(fn,ft) 
                        os.rmdir(fo)
                if not booFound:
                    print "*couldnt find file"
                    
    
        else:
            print "couldnt find file  ",found
    

mkdir: cannot create directory ‘/home/some/an/’: File exists
1 -  03-13-2016 Part 2-2\Powerone - Hell Hospital [MF] (pdf).rar powerone hell hospital [mf]


In [89]:
dr='/media/some/sam/download-ep/'
lst=!find "$dr" -type f -iname "*.epub"
len(lst)

4675

In [101]:
def searchdb(seTerm,db):
    seTerm=[se.strip() for se in seTerm.split(' ')]
    allres=[list
            (filter
             (lambda x: any(xx.lower().replace("'","").count(se.strip().lower()) > 0 for xx in x[3:7]  ), db)) 
            for se in seTerm]
    #find common items in multiple lists
    return reduce(lambda sx, sy: filter(lambda x: x in sx,sy), ((i) for i in allres))

In [ ]:
rmlst=[]
t0 = time.time()
for f in lst:
    #print 'Full Name: ',f,'\n'

    fp,fn=os.path.split(f)
    ext=re.findall('.+(\..+)',fn)[0]
    fn=fn.replace('_'," ").replace(ext,'')
    fn=''.join(fnn for fnn in fn if fnn in string.letters+" ")
    #print 'File Name: ',fn,'\n'
    #print 'Full Folder Name: ',fp,'\n'
    try:
        fpr=re.findall('.*/(.+)',fp)[0].replace('_'," ")
        #print 'Parent Folder Name:',fpr,'\n'
    except:
        print "error"
   

    found=searchdb(fn,dBB)
    if len(found)>0:
            rmlst.append([f,found])

t1 = time.time()
total = t1-t0
print("files to delete",len(rmlst))
print('time: ',total)
print('resources: ',resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1000)

In [197]:
found

In [188]:
strg=!ebook-meta "$f"

In [192]:
stdict={st.split("  : ")[0].strip(): st.split("  : ")[1].strip() for st in strg if st.count("  : ")>0}

In [223]:
stdict['Author(s)']+" "+stdict['Title']

"Larry Niven & Brenda Cooper [Niven, Larry] Building Harlequin's Moon"

In [224]:
ffd=stdict['Published'][:stdict['Published'].index('T')]
yr=ffd[0:4]
dd=ffd[-2:]
mm=ffd[5:7]
print fn
fnd="*"+yr+"*.rar"
#print fnd
fls=findr(fnd,'/media/some/sam/dall/')
if fls>0:
    for flss in fls:
        rf = rarfile.RarFile(flss)
        for fll in rf.infolist():
                #print fll.filename.split('\\')[1],fn
                if fll.filename.count("\\")>0 and similarity(fll.filename.split('\\')[1],fn)>0.3:
                    print fll.filename

Building Harlequins Moon - Larry Niven  Brenda Cooper
New Books 04-29-2010\Larry Niven - Handicap (html).rar


In [ ]:

fls

In [163]:
re.findall('^Published:(.+)',strg)

[]

In [133]:
fp

'Caught Up in Her (Caught Up In Love 0.5.epub'